# Load Data

In [1]:
import pandas as pd
df = pd.read_csv("hotel_bookings.csv")

In [2]:
from sklearn import preprocessing
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

le = preprocessing.LabelEncoder()

# Split the features and labels
X = df.loc[:, df.columns != 'is_canceled']
y = df.is_canceled
# Change the data into one-hot encoding (for features) and change label to 0-1
X = pd.get_dummies(X)
y = le.fit_transform(y)

# Model

## Example 1

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def init_params(layer_dims):
    np.random.seed(3)
    params = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        params['W'+str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*0.01
        params['b'+str(l)] = np.zeros((layer_dims[l], 1))
        
    return params

In [5]:
# Z (linear hypothesis) - Z = W*X + b , 
# W - weight matrix, b- bias vector, X- Input 

def sigmoid(Z):
    A = 1/(1+np.exp(np.dot(-1, Z)))
    cache = (Z)
    
    return A, cache

In [6]:
def forward_prop(X, params):
    
    A = X # input to first layer i.e. training data
    caches = []
    L = len(params)//2
    for l in range(1, L+1):
        A_prev = A
        
        # Linear Hypothesis
        Z = np.dot(params['W'+str(l)], A_prev) + params['b'+str(l)] 
        
        # Storing the linear cache
        linear_cache = (A_prev, params['W'+str(l)], params['b'+str(l)]) 
        
        # Applying sigmoid on linear hypothesis
        A, activation_cache = sigmoid(Z) 
        
         # storing the both linear and activation cache
        cache = (linear_cache, activation_cache)
        caches.append(cache)
    
    return A, caches

In [7]:
def cost_function(A, Y):
    m = Y.shape[1]
    
    cost = (-1/m)*(np.dot(np.log(A), Y.T) + np.dot(log(1-A), 1-Y.T)) 
    
    return cost

In [8]:
def one_layer_backward(dA, cache):
    linear_cache, activation_cache = cache
    
    Z = activation_cache
    dZ = dA*sigmoid(Z)*(1-sigmoid(Z)) # The derivative of the sigmoid function
    
    A_prev, W, b = linear_cache
    m = A_prev.shape[1]
    
    dW = (1/m)*np.dot(dZ, A_prev.T)
    db = (1/m)*np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [9]:
def backprop(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = -(np.divide(Y, AL) - np.divide(1-Y, 1-AL))
    
    current_cache = caches[L-1]
    grads['dA'+str(L-1)], grads['dW'+str(L-1)], grads['db'+str(L-1)] = one_layer_backward(dAL, current_cache)
    
    for l in reversed(range(L-1)):
        
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = one_layer_backward(grads["dA" + str(l+1)], current_cache)
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
    return grads

In [10]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2
    
    for l in range(L):
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] -learning_rate*grads['W'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] -  learning_rate*grads['b'+str(l+1)]
        
    return parameters

In [11]:
def train(X, Y, layer_dims, epochs, lr):
    params = init_params(layer_dims)
    cost_history = []
    
    for i in range(epochs):
        Y_hat, caches = forward_prop(X, params)
        cost = cost_function(Y_hat, Y)
        cost_history.append(cost)
        grads = backprop(Y_hat, Y, caches)
        
        params = update_parameters(params, grads, lr)
        
        
    return params, cost_history

In [15]:
train(X,y,layer_dims, 5,1e05)

NameError: name 'layer_dims' is not defined

## Example 2 

In [19]:
def train_model(X_train, X_test, y_train, y_test):
  model = tf.keras.models.Sequential([
            tf.keras.Input(shape=(1186), dtype='float32'),
            tf.keras.layers.Dense(units=1024, activation='relu'),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(units=1, activation='sigmoid')
          ])
  model.compile(optimizer=Adam(lr=0.0001),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  # Callback to reduce learning rate if no improvement in validation loss for certain number of epochs
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-8, verbose=0)
# Callback to stop training if no improvement in validation loss for certain number of epochs
  early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0)
  history = model.fit(
              X_train, y_train,
              epochs=1000,
              validation_data=(X_test, y_test),
              callbacks=[reduce_lr, early_stop],
              batch_size = 3,
              verbose=0
            )
  tr_loss, tr_acc = model.evaluate(X_train, y_train)
  loss, accuracy = model.evaluate(X_test, y_test)
  return model, history, tr_loss, tr_acc, loss, accuracy

In [20]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [21]:
from sklearn.model_selection import KFold
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam # - Works
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping


kfold = KFold(n_splits=5, random_state=42, shuffle=True)
loss_arr = []
acc_arr = []
trloss_arr = []
tracc_arr = []
temp_acc = 0
for train, test in kfold.split(df):
  model, history, trloss_val, tracc_val, loss_val, acc_val =   train_model(X.iloc[train], X.iloc[test], y[train], y[test])
  # If we got better accuracy in validation, then we save the split scenario and the model
  if acc_val > temp_acc:
      print("Model changed")
      temp_acc = acc_val
      model.save('best_model.h5')
      train_index = train
      test_index = test
      best_history = history
  trloss_arr.append(trloss_val)
  tracc_arr.append(tracc_val)
  loss_arr.append(loss_val)
  acc_arr.append(acc_val)

/home/ggnicolau/miniconda3/envs/jupyter-1/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-04-09 00:19:06.849735: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 906217856 exceeds 10% of free system memory.


KeyboardInterrupt: 

## Example 3

In [27]:
model = keras.Sequential([keras.layers.Flatten(batch_size = 3, input_shape = (None, 1186)),
                keras.layers.Dense(128,activation = tf.nn.sigmoid),                          
                keras.layers.Dense(10,activation = tf.nn.softmax)])
model.compile(optimizer = 'adam',loss='sparse_categorical_crossentropy',metrics =['accuracy'])

ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (3, None)

In [26]:
model.fit(X, y,epochs = 5)

Epoch 1/5


2022-04-09 00:20:39.772789: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1132772320 exceeds 10% of free system memory.


ValueError: in user code:

    File "/home/ggnicolau/miniconda3/envs/jupyter-1/lib/python3.10/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/ggnicolau/miniconda3/envs/jupyter-1/lib/python3.10/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ggnicolau/miniconda3/envs/jupyter-1/lib/python3.10/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/ggnicolau/miniconda3/envs/jupyter-1/lib/python3.10/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/ggnicolau/miniconda3/envs/jupyter-1/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/ggnicolau/minicond

# Example 4

In [3]:
#standardizing the input feature
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X

array([[ 2.22705112e+00, -1.63476794e+00, -1.21405347e-02, ...,
        -5.78833016e-03, -2.89412872e-03, -4.09293323e-03],
       [ 5.92338470e+00, -1.63476794e+00, -1.21405347e-02, ...,
        -5.78833016e-03, -2.89412872e-03, -4.09293323e-03],
       [-9.07814066e-01, -1.63476794e+00, -1.21405347e-02, ...,
        -5.78833016e-03, -2.89412872e-03, -4.09293323e-03],
       ...,
       [-6.55153290e-01,  1.19219514e+00,  5.75875059e-01, ...,
        -5.78833016e-03, -2.89412872e-03, -4.09293323e-03],
       [ 4.66822005e-02,  1.19219514e+00,  5.75875059e-01, ...,
        -5.78833016e-03, -2.89412872e-03, -4.09293323e-03],
       [ 9.45031628e-01,  1.19219514e+00,  5.75875059e-01, ...,
        -5.78833016e-03, -2.89412872e-03, -4.09293323e-03]])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
from keras import Sequential
from keras.layers import Dense

2022-04-09 00:25:49.160680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-09 00:25:49.160712: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(1186, activation='relu', kernel_initializer='random_normal', input_dim=1186))
#Second  Hidden Layer
classifier.add(Dense(1186, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

2022-04-09 00:25:52.785836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-09 00:25:52.785871: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-09 00:25:52.785897: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (brspobitanl1727): /proc/driver/nvidia/version does not exist
2022-04-09 00:25:52.786457: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [8]:
import tensorflow as tf

#Fitting the data to the training dataset
#X_train = tf.expand_dims(X_train, axis=-1)
# X_train = X_train.reshape(None, 8)
# #y_train = tf.expand_dims(y_train, axis=-1)
# y_train = y_train.reshape(None, 8)
classifier.fit(X_train,y_train, batch_size=100, epochs=5)

Epoch 1/5


2022-04-09 00:26:00.324393: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 396470312 exceeds 10% of free system memory.


836/836 [==============================] - 26s 31ms/step - loss: nan - accuracy: 0.6292
Epoch 2/5
701/836 [========================>.....] - ETA: 4s - loss: nan - accuracy: 0.6287

KeyboardInterrupt: 

In [14]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

   1/2612 [..............................] - ETA: 6:39 - loss: nan - accuracy: 0.6250

2022-04-09 00:24:18.911004: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 396470312 exceeds 10% of free system memory.


2612/2612 [==============================] - 4s 1ms/step - loss: nan - accuracy: 0.6291


[nan, 0.6291146874427795]

In [15]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[22589     0]
 [13228     0]]
